In [1]:
import pandas as pd

In [2]:
import numpy as np

## Part A : Data Massage

### 1. Declare input file names.

File needs to be in CSV format for faster loading.

#### Input 1 = ME2L_W01.CSV
#### Input 2 = ME2L_W02.CSV

Place the 2 input files in the current directory as the script file.

In [3]:
# import time
# start = time.time()
# me2l_w01_file = "test/me2l_w01x.xlsx"
# df1 = pd.read_excel(me2l_w01_file,sheet_name='Raw');
# end = time.time()
# print(end - start)

In [4]:
me2l_w01_file = "test/me2l_w01.csv"
me2l_w02_file = "test/me2l_w02.csv"

### 2. Read input files

If file contains large data sets with alot of columns, reading will take a bit of time.

In [5]:
# Load first input file
df1 = pd.read_csv(me2l_w01_file,parse_dates=True, encoding='ISO-8859-1');
# df1.head()

C:\Users\TM36250\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (5,9,12,14) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [6]:
# Load second input file
df2 = pd.read_csv(me2l_w02_file,parse_dates=True);
# df2.head()

### 3. Extract PO with correct CAPEX Category from ME2L_W02 

Capex Category = (A, N, P, X).
But how to check if the project is deployment,operation related ? Any keyword or vendor as filters ?

In [7]:
capex_w02 = df2[(df2['Acct Assignment Cat.'] == 'A') |
                (df2['Acct Assignment Cat.'] == 'N') |
                (df2['Acct Assignment Cat.'] == 'P') |
                (df2['Acct Assignment Cat.'] == 'X')]

### 4. Append extraction from ME2L_W02 to ME2L_W01

Data will be pasted at the bottom of ME2L_W01 data set.

In [8]:
df1.append(capex_w02);

### 5. Remove unwanted columns.

This will allow processing of data faster since unwanted data is discarded.Columns removed:
- Deletion Indicator
- Req. Tracking Number

In [9]:
df1.drop(columns=['Deletion Indicator','Req. Tracking Number'],inplace=True);

### 6. Insert 'Vendor Code' and 'Vendor' column at the begining.

Extract Vendor Code info and Vendor Name info from 'Vendor/supplying plant' column.

Based on old SAP entries there are multiple Vendor Code entries which are similar but Vendor name spelling is different. Example:
- Vendor Code = 1107060 , Vendor Name = Corporation A Sdn. Bhd.
- Vendor Code = 1107060 , Vendor Name = Corporation A Technologies


Need to rearrange all PO so that it is assigned to one unique Vendor Code and one Vendor Name. So that in final summary there are no multiple vendor name entries that refers to a single vendor.

Once this process done, remove the 'Vendor/supplying plant' column since it is redundant.

In [10]:
#Extract Vendor code and Vendor name and insert as new columns
extract_vendor = df1["Vendor/supplying plant"].str.split(" ", n = 1, expand = True) 
df1.insert(0,'Vendor Code', extract_vendor[0]);
df1.insert(1,'Vendor', extract_vendor[1]);

#Create a separate dataframe for vendor code and vendor name
vendor_list={'Vendor Code': extract_vendor[0],'Vendor Unique': extract_vendor[1]}
df_vendor = pd.DataFrame(vendor_list, columns = ['Vendor Code', 'Vendor Unique'])

#Remove duplicate entries in vendor dataframe
df_vendor.drop_duplicates(inplace=True)
#Remove duplicates in vendor code but having multiple versions of vendor name. Only keep the first one found. 
#There will only be one 1x vendor code tied to 1x vendor name.
df_vendor.drop_duplicates(subset='Vendor Code',keep='first',inplace=True)
#Sort the values in ascending manner
df_vendor.sort_values(by=['Vendor Code'],inplace=True)

#Vlookup between both df1 and df_vendor to streamline the vendor name.
df1 = pd.merge(df1,df_vendor,on ='Vendor Code',how ='inner')
#Drop the initial Vendor column and Vendor/supplying plant
df1.drop(columns=['Vendor'],inplace=True)
df1.drop(columns=['Vendor/supplying plant','Outline Agreement'],inplace=True)

#Rename Vendor name columns
df1.rename(columns={'Vendor Unique': 'Vendor'}, inplace=True)

#Rearrange columns to place vendor name in 2nd position.
df1 = df1[['Vendor Code','Vendor', 'Purchasing Document', 'Short Text', 'Currency',
       'Acct Assignment Cat.', 'Document Date', 'Order Quantity', 'Net price',
       'Net Order Value', 'Still to be delivered (qty)',
       'Still to be delivered (value)', 'Still to be invoiced (qty)',
       'Still to be invoiced (val.)']]

### 7. Classify each PO according to CAPEX/OPEX category.

Use values from 'Acc Assignment Cat.' as reference for classification.
- CAPEX (A, N, P, X)
- OPEX (F, K, Blank)

Add a new column called 'Capex/Opex' at the end of the data set.

In [11]:
#Add Capex/Opex Category
df1.loc[ (df1['Acct Assignment Cat.'] == 'A') |
        (df1['Acct Assignment Cat.'] == 'N') |
        (df1['Acct Assignment Cat.'] == 'P') |
        (df1['Acct Assignment Cat.'] == 'X'),'Capex/Opex'] = 'CAPEX'

df1.loc[ (df1['Acct Assignment Cat.'] == 'F') |
        (df1['Acct Assignment Cat.'] == 'K') |
        (df1['Acct Assignment Cat.'] == ''),'Capex/Opex'] = 'OPEX'

### 8. Calculate Aging, PO Category & PO Year

Add 2 new columns to input the Aging info:
- 'Aging' = shows aging in number of days
- 'Aging (Months & Days)' = shows aging in number of months and remaining days

Check PO Category if the value in 'Document Date' column is similar to current year. If not then PO < Current Year

Check PO Year using the 'Document Date' column. Extract year info.

In [12]:
import datetime

import time
start = time.time()

def calculate_aging_days(doc_date,date_now):
    po_date = datetime.datetime.strptime(doc_date, '%d/%m/%Y')
    delta = now - po_date
    return int(delta.days)

def calculate_aging_months_days(aging_days):
    months = int(aging_days/30)
    remaining_days = int(aging_days%30)
    return str(months) + ' months ' + str(remaining_days) + ' days'

def assign_po_year(doc_date):
    po_date = datetime.datetime.strptime(doc_date, '%d/%m/%Y')
    return po_date.year

def assign_po_category(doc_date,current_year):
    po_date = datetime.datetime.strptime(doc_date, '%d/%m/%Y')
    return 'PO ' + str(now.year) if  po_date.year == now.year else 'PO <' + str(now.year)

now = datetime.datetime.now()
df1['Aging Days'] = df1.apply(lambda row: calculate_aging_days(row['Document Date'],now),axis=1)
# df1['PO Year'] =  datetime.datetime.now().year
df1['PO Year'] = df1.apply(lambda row: assign_po_year(row['Document Date']),axis=1)
df1['PO Category'] = df1.apply(lambda row: assign_po_category(row['Document Date'],datetime.datetime.now().year),axis=1)
df1['Aging (Months & Days)'] = df1.apply(lambda row: calculate_aging_months_days(row['Aging Days']),axis=1)

# df1.at[i,'PO Year'] = po_date.year\n",
#     "    # Determine PO Category\n",
#     "    df1.at[i,'PO Category'] = 'PO ' + str(now.year) if  po_date == now.year else 'PO <' + str(now.year)\n"

end = time.time()
print(end - start)

19.40033984184265


### 9. Calculate Aging Category

Classify each PO into the following categories:
- ( <6 Months )
- ( >6 Months )
- ( >18 Months )

Add a new column 'Aging Category' at the end.

In [13]:

df1.loc[ (df1['Aging Days'] <= 182),'Aging Category'] = '<6 Months'
df1.loc[ (df1['Aging Days'] > 182) & (df1['Aging Days'] < 540),'Aging Category'] = '>6 Months'
df1.loc[ (df1['Aging Days'] > 540),'Aging Category'] = '>18 Months'
print('done')


done


### 10. Assign GR status

Status can either be 'Open' or 'Closed' depending on there are still value to be delivered. Use the column 'Still to be delivered (value)' as reference.

In [14]:

df1['Still to be delivered (value)'] = df1['Still to be delivered (value)'].str.replace(',','')
df1['Still to be delivered (value)'] = df1['Still to be delivered (value)'].astype(float)
df1.loc[ (df1['Still to be delivered (value)'] > 0),'GR Status'] = 'Open'
df1.loc[ (df1['Still to be delivered (value)'] == 0),'GR Status'] = 'Closed'
print('done')


done


### 11. Assign GRIR Status
Status can be either 'Open' or 'Closed' depending on value in column 'Still to be invoiced (val.)'

In [15]:
df1['Still to be invoiced (val.)'] = df1['Still to be invoiced (val.)'].str.replace(',','')
df1['Still to be invoiced (val.)'] = df1['Still to be invoiced (val.)'].astype(float)
df1['GRIR Status'] = df1['Still to be invoiced (val.)'].apply(lambda x: 'Open' if x > -0.1 and x < 0.1 else 'Closed')

print('done')

done


### 12. Assign PO Category


In [16]:
# df1.loc[ (df1['GR Status'] == 'Closed') & (df1['GRIR Status'] == 'Closed'),'PO Status'] = 'Closed'
# df1.loc[ (df1['GR Status'] == 'Open') | (df1['GRIR Status'] == 'Open'),'PO Status'] = 'Open'

df1['PO Status'] = np.where((df1['GR Status'] == 'Closed') & (df1['GRIR Status'] == 'Closed'), 'Closed', 'Open')
print('done')

done


### 13. Calculate Still to be delivered (MYR-Value)


In [17]:
# Set USD Currency rate here.
usd_currency_rate = 4.1

df1['Still to be delivered (MYR-Value)'] = np.where(df1['Currency']== 'USD', df1['Still to be delivered (value)']*usd_currency_rate, df1['Still to be delivered (value)'])
print('done')

done


## Part B : Prepare Report

### 1. Write data to excel file

In [18]:
output_filename = 'output/output_me2l.xlsx'
writer = pd.ExcelWriter(output_filename, engine='xlsxwriter')
start = time.time()


# df_final.to_excel(output_filename, sheet_name='data', engine='xlsxwriter',index=False)

# df1.to_excel(output_filename, sheet_name='data', engine='xlsxwriter',index=False)
df1.to_excel(writer, sheet_name='data',index=False)
print('done')
end = time.time()
print(end - start)

done
60.08488488197327


In [19]:
# start = time.time()
# df1.to_csv('output/final_csv.csv', index=False)
# end = time.time()
# print(end - start)

## Create PO Summary List


In [20]:
df1['Net Order Value'] = df1['Net Order Value'].str.replace(',','')
df1['Net Order Value']= df1['Net Order Value'].astype(float)

In [21]:
df2 = df1.groupby(['Vendor','Purchasing Document','Capex/Opex', 'Currency','Document Date']).agg('sum')
df2.drop(columns=['Aging Days','PO Year'],inplace=True)

In [22]:
start = time.time()
df2.to_excel(writer, sheet_name='PO',merge_cells=False)
writer.save()
end = time.time()
print(end - start)

34.275941133499146


In [23]:
# df2.to_excel('output/df2.xlsx',sheet_name='PO',index=False)

In [24]:
df2.head(100)

Net Order Value  \
Vendor                                 Purchasing Document Capex/Opex Currency Document Date                    
     Fibrecomm Network (M) Sdn Bhd     4901344491          OPEX       MYR      21/6/2018             10000.00   
     TELEKOM MALAYSIA BERHAD           4800361190          OPEX       MYR      8/11/2019              4250.00   
     VADS BERHAD                       4800347623          OPEX       MYR      11/10/2018            21000.00   
                                       4800347911          OPEX       MYR      18/10/2018            34080.00   
                                       4800348920          OPEX       MYR      14/11/2018             4500.00   
...                                                                                                       ...   
   C & C UNIVERSAL ENGINEERING SDN. BH 4901604885          CAPEX      MYR      14/7/2020             10530.15   
                                       4901604887          CAPEX      MYR      14/7/2020             10530.15   
                                       4901604888          CAPEX      MYR      14/7/2020             10530.15   
                                       4901604890          CAPEX      MYR      14/7/2020             10530.15   
                                       4901604891          CAPEX      MYR      14/7/2020             10530.15   

                                                                                              Still to be delivered (value)  \
Vendor                                 Purchasing Document Capex/Opex Currency Document Date                                  
     Fibrecomm Network (M) Sdn Bhd     4901344491          OPEX       MYR      21/6/2018                               0.00   
     TELEKOM MALAYSIA BERHAD           4800361190          OPEX       MYR      8/11/2019                               0.00   
     VADS BERHAD                       4800347623          OPEX       MYR      11/10/2018                              0.00   
                                       4800347911          OPEX       MYR      18/10/2018                              0.00   
                                       4800348920          OPEX       MYR      14/11/2018                              0.00   
...                                                                                                                     ...   
   C & C UNIVERSAL ENGINEERING SDN. BH 4901604885          CAPEX      MYR      14/7/2020                               0.00   
                                       4901604887          CAPEX      MYR      14/7/2020                           10530.15   
                                       4901604888          CAPEX      MYR      14/7/2020                           10530.15   
                                       4901604890          CAPEX      MYR      14/7/2020                               0.00   
                                       4901604891          CAPEX      MYR      14/7/2020                           10530.15   

                                                                                              Still to be invoiced (val.)  \
Vendor                                 Purchasing Document Capex/Opex Currency Document Date                                
     Fibrecomm Network (M) Sdn Bhd     4901344491          OPEX       MYR      21/6/2018                             0.00   
     TELEKOM MALAYSIA BERHAD           4800361190          OPEX       MYR      8/11/2019                             0.00   
     VADS BERHAD                       4800347623          OPEX       MYR      11/10/2018                            0.00   
                                       4800347911          OPEX       MYR      18/10/2018                            0.00   
                                       4800348920          OPEX       MYR      14/11/2018                            0.00   
...                                                                                              